# Create an interface with strain matching

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to select an interface between two materials with minimal strain.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate interfaces with strain matcher and plot strain vs number of atoms 
1. Select the interface with the desired strain and visualize it

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered.
1. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. 
1. ZSL strain matching is performed using Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [57]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Maximum area for the superlattice search algorithm
MAX_AREA = 350

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [58]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [83]:
import numpy as np
from mat3ra.made.tools.build.supercell import create_supercell
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[1]
film = materials[1]


def find_optimal_multiple(angle: float, max_multiple: int = 100, relative_error_tol: float = 0.01) -> int:
    """
    Find the optimal multiple that minimizes the relative error for a given angle.

    Args:
        angle (float): The desired angle in degrees.
        max_multiple (int): The maximum multiple to consider.

    Returns:
        int: The optimal multiple.
    """
    optimal_multiple = 1
    min_relative_error = float('inf')

    for multiple in range(1, max_multiple + 1):
        n = np.floor(np.sin(np.deg2rad(angle)) * multiple)
        m = np.floor(np.cos(np.deg2rad(angle)) * multiple)
        computed_angle = np.arcsin(n / multiple)
        relative_error = np.abs(angle - np.rad2deg(computed_angle)) / angle

        if relative_error < min_relative_error:
            min_relative_error = relative_error
            optimal_multiple = multiple
            
        if relative_error < relative_error_tol:
            break

    return optimal_multiple

# Example usage
angle = 35  # degrees
optimal_multiple = find_optimal_multiple(angle, 50)
n = np.floor(np.sin(np.deg2rad(angle)) * optimal_multiple)
m = np.floor(np.cos(np.deg2rad(angle)) * optimal_multiple)
computed_angle = np.arcsin(n / optimal_multiple)
relative_error = np.abs(angle - np.rad2deg(computed_angle)) / angle
print(f"Optimal multiple: {optimal_multiple}")
print(f"Computed angle: {np.rad2deg(computed_angle)}")
print(f"Relative error: {relative_error * 100:.2f}%")

rotation_matrix = [
    [n, -m, 0],
    [m, n, 0],
    [0, 0, 1]
]
print(f"Rotation matrix: {rotation_matrix}")

rotated_film = create_supercell(film, rotation_matrix)

0: Data from 0-Ni has been read successfully.
1: Data from 1-Graphene has been read successfully.
2: Data from 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1) has been read successfully.
3: Data from 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133 has been read successfully.
4: Data from 12-Cd4 Te4 has been read successfully.
5: Data from 13-Si4 C4 has been read successfully.
6: Data from 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804 has been read successfully.
7: Data from 4-Te2Mo has been read successfully.
8: Data from 5-HfO2 has been read successfully.
9: Data from 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0] has been read successfully.
10: Data from 7-Ag4 has been read successfully.
11: Data from 8-Si, Silicene, HEX (P-3m1) 2D (Monolayer), 2dm-5934 has been read successfully.
12: Data from 9-GaAs, Gallium Arsenide, FCC (F-43m) 3D (Bulk), mp-2534 has been read successfully.
13: Data from C196 Twisted Interface has been read successfully.
14

### 1.4. Preview Substrate and Film

In [84]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film, rotated_film], repetitions=[1, 1, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='C2 - Material - rotation: 0x', layout=Layout(align_self='center'…

In [85]:
from pymatgen.transformations.standard_transformations import RotationTransformation
from utils.jupyterlite import get_data
from mat3ra.made.tools.convert import to_pymatgen, from_pymatgen
import numpy as np
from typing import Tuple, List
from pymatgen.core import Structure, Lattice
from pydantic import BaseModel, Field

class TwistedInterfaceConfiguration(BaseModel):
    substrate: Structure
    film: Structure
    twist_angle: float
    max_strain: float = 0.1
    max_supercell_size: int = 10

def rotation_matrix_2d(angle: float) -> np.ndarray:
    """Create a 2D rotation matrix for the given angle in degrees."""
    theta = np.radians(angle)
    return np.array([[np.cos(theta), -np.sin(theta)],
                     [np.sin(theta), np.cos(theta)]])

def find_matching_vectors(a1: np.ndarray, a2: np.ndarray, b1: np.ndarray, b2: np.ndarray, 
                          max_size: int, max_strain: float) -> Tuple[np.ndarray, np.ndarray, float]:
    best_strain = float('inf')
    best_m, best_n, best_p, best_q = 0, 0, 0, 0

    for m in range(1, max_size + 1):
        for n in range(max_size + 1):
            substrate_vector = m * a1 + n * a2
            for p in range(1, max_size + 1):
                for q in range(max_size + 1):
                    film_vector = p * b1 + q * b2
                    strain = np.linalg.norm(substrate_vector - film_vector) / np.linalg.norm(substrate_vector)
                    if strain < best_strain and strain <= max_strain:
                        best_strain = strain
                        best_m, best_n, best_p, best_q = m, n, p, q

    if best_strain == float('inf'):
        raise ValueError("No matching vectors found within strain limit")

    return (np.array([best_m, best_n]), np.array([best_p, best_q]), best_strain)

def create_bilayer_twisted_interface(structure: Structure, twist_angle: float, 
                                     max_size: int = 10, max_strain: float = 0.1, 
                                     layer_separation: float = 3.0) -> Structure:
    # Create a copy for the film layer
    film = structure.copy()
    
    # Rotate the film layer
    rotation = RotationTransformation([0, 0, 1], twist_angle)
    rotated_film = rotation.apply_transformation(film)

    # Extract 2D lattice vectors
    a1, a2 = structure.lattice.matrix[:2, :2].T
    b1, b2 = rotated_film.lattice.matrix[:2, :2].T

    # Find matching vectors
    (m, n), (p, q), strain = find_matching_vectors(a1, a2, b1, b2, max_size, max_strain)

    # Create supercells
    substrate_supercell_matrix = np.array([[m, n, 0], [-n, m, 0], [0, 0, 1]])
    film_supercell_matrix = np.array([[p, q, 0], [-q, p, 0], [0, 0, 1]])

    substrate_supercell = structure.copy()
    substrate_supercell.make_supercell(substrate_supercell_matrix)

    film_supercell = rotated_film.copy()
    film_supercell.make_supercell(film_supercell_matrix)

    # Adjust lattice to make them match exactly
    new_lattice = substrate_supercell.lattice

    # Combine layers
    c_substrate = np.max(substrate_supercell.cart_coords[:, 2])
    c_film = np.max(film_supercell.cart_coords[:, 2])
    c_total = c_substrate + c_film + layer_separation

    combined_lattice = Lattice.from_parameters(
        *new_lattice.abc[:2], c_total,
        *new_lattice.angles
    )

    combined_structure = Structure(combined_lattice, [], [])
    
    # Add substrate layer
    for site in substrate_supercell:
        combined_structure.append(site.species, site.frac_coords, properties=site.properties)

    # Add film layer
    for site in film_supercell:
        new_coords = site.frac_coords
        new_coords[2] = (site.coords[2] + c_substrate + layer_separation) / c_total
        combined_structure.append(site.species, new_coords, properties=site.properties)

    return combined_structure

# Usage
substrate_struct = to_pymatgen(substrate)
film_struct = to_pymatgen(film)
config = TwistedInterfaceConfiguration(substrate=substrate_struct, film=film_struct, twist_angle=1.1)
twisted_interface = from_pymatgen(create_bilayer_twisted_interface(config.substrate, config.twist_angle))

# Visualize the twisted interface
visualize([Material(twisted_interface)], repetitions=[1, 1, 1])
visualize([Material(twisted_interface)], repetitions=[1, 1, 1], rotation="-90x")

twisted_interface["name"] += f" Twisted Interface {config.twist_angle} degrees"

set_data("materials_out", [Material(twisted_interface).to_json()])

GridBox(children=(VBox(children=(Label(value='C196 - Material - rotation: 0x,0y,0z', layout=Layout(align_self=…

GridBox(children=(VBox(children=(Label(value='C196 - Material - rotation: -90x', layout=Layout(align_self='cen…

Data for materials_out written to uploads/C196 Twisted Interface 1.1 degrees.json


In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=(0, 0, 1),
    thickness=1, # in atomic layers
    vacuum=0, # in atomic layers
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=(0,0,1),
    thickness=1, # in atomic layers
    vacuum=1, # in atomic layers
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

### 2.2. Get possible terminations for the slabs

In [ ]:
film_slab_terminations = get_terminations(film_slab_configuration)
substrate_slab_terminations = get_terminations(substrate_slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination) for termination in substrate_slab_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in film_slabs ], repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_slabs ], repetitions=[3, 3, 1], rotation="-90x")  

### 2.4. Print terminations for the interface

In [ ]:
from itertools import product

termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))    
print("Termination Pairs (Film, Substrate)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

### 2.5. Select termination pair for the interface

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0

termination_pair = termination_pairs[TERMINATION_PAIR_INDEX]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

film_termination, substrate_termination = termination_pair
interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=3.0, # in Angstrom
    vacuum=20.0 # in Angstrom
)

### 3.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
from mat3ra.made.tools.analyze import get_surface_area
# rotated_film_area = get_surface_area(film)
# MAX_AREA = rotated_film_area * 1.5
print(f"Max area: {MAX_AREA}")
zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA,
    angle_tol= 0.001,
)

### 3.3. Generate interfaces with strain matcher
Interfaces are sorted by size and strain.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters

matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(strain_matching_parameters=zsl_strain_matching_parameters))

interfaces_sorted_by_size_and_strain= matched_interfaces_builder.get_materials(configuration=interface_configuration)

### 3.4. Plot interfaces by size and strain


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_sorted_by_size_and_strain, PLOT_SETTINGS)

### 3.5. Select the interface

Select the index for the interface with the lowest strain and the smallest number of atoms.

In [ ]:
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0,1)
selected_interfaces = interfaces_sorted_by_size_and_strain[interfaces_slice_range_or_index]

## 4. Preview the selected material

In [ ]:
visualize(selected_interfaces, repetitions=[3, 3, 1])
visualize(selected_interfaces, repetitions=[3, 3, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_data

materials_as_json = [selected_interface.to_json() for selected_interface in selected_interfaces]
set_data("materials", materials_as_json)

In [ ]:
from mat3ra.made.tools.build.interface.builders import TwistedInterfaceConfiguration, TwistedInterfaceBuilder, \
    TwistedInterfaceBuilderParameters

# Step 2: Create SlabConfigurations for substrate and film
substrate_config = SlabConfiguration(
    bulk=substrate,
    miller_indices=(0,0,1),
    thickness=1
)

film_config = SlabConfiguration(
    bulk=film,
    miller_indices=(0,0,1),
    thickness=1
)

# Step 3: Create a TwistedInterfaceConfiguration
twisted_config = TwistedInterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance_z=2.0,
    vacuum=10.0,
    twist_angle=30.0  
)

# Step 4: Create TwistedInterfaceBuilderParameters (optional, using defaults here)
builder_params = TwistedInterfaceBuilderParameters(strain_matching_parameters=zsl_strain_matching_parameters)

# Step 5: Create the TwistedInterfaceBuilder
builder = TwistedInterfaceBuilder(build_parameters=builder_params)
                                  

# Step 6: Generate the twisted interfaces
twisted_interfaces = builder.get_materials(twisted_config)

# Step 7: Analyze the results
print(f"Generated {len(twisted_interfaces)} twisted interfaces")
for i, interface in enumerate(twisted_interfaces):
    print(f"\nInterface {i + 1}:")
    print(f"Name: {interface.name}")
    print(f"Number of atoms: {len(interface.basis.elements.values)}")
    print(f"Lattice parameters: a={interface.lattice.a:.3f}, b={interface.lattice.b:.3f}, c={interface.lattice.c:.3f}")
    print(f"Mean absolute strain: {interface.metadata.get('interface_properties').get('mean_abs_strain', 'N/A')}")
    print(f"Twist angle: {interface.metadata['build']['configuration']['twist_angle']:.2f} degrees")


In [ ]:
from utils.jupyterlite import set_data
selected_twisted_interfaces=twisted_interfaces[slice(74,78)]
visualize(selected_twisted_interfaces, repetitions=[1, 1, 1])
for i,material in enumerate(selected_twisted_interfaces):
    material.name += f" {i}"
set_data("materials_out", [i.to_json() for i in selected_twisted_interfaces])
twisted_interfaces[0].metadata
